In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetB0, efficientnet_v2, EfficientNetB1
from keras.models import Sequential, load_model, Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback, EarlyStopping
from keras.optimizers import Adam, Adagrad, SGD
import matplotlib.pyplot as plt
from keras import backend as K
from keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import math
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
import seaborn as sns
import tensorflow_addons as tfa

In [2]:
# Paths to the CSV files (Update these paths)
train_csv_file = "/home/halwa/fyp/Training/train2.csv"
validation_csv_file = "/home/halwa/fyp/Training/validation2.csv"

# Define batch size and number of epochs
batch_size = 32

# Create an ImageDataGenerator with augmentation for training
trainAug = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Create a generator for loading and preprocessing training images with augmentation
trainGen = trainAug.flow_from_dataframe(
    dataframe=pd.read_csv(train_csv_file, encoding='ISO-8859-1'),
    x_col='Image_Path',
    y_col='Species',
    target_size=(240, 240),
    batch_size=batch_size,
    class_mode='categorical',
)



# Create a generator for loading and preprocessing validation images in smaller batches
valAug = ImageDataGenerator(rescale=1.0/255.0)
valGen = valAug.flow_from_dataframe(
    dataframe=pd.read_csv(validation_csv_file, encoding='ISO-8859-1'),
    x_col='Image_Path',
    y_col='Species',
    target_size=(240, 240),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 24061 validated image filenames belonging to 588 classes.
Found 6145 validated image filenames belonging to 588 classes.


In [4]:
# Get class indices
class_indices = trainGen.class_indices

# Create a DataFrame from class indices
df = pd.DataFrame(list(class_indices.items()), columns=['Class', 'Index'])

# Specify the path to the CSV file
csv_file_path = '/home/halwa/fyp/Training/class_indices.csv'

# Write class indices to the CSV file
df.to_csv(csv_file_path, index=False)

In [3]:
train_csv_file = "/home/halwa/fyp/Training/train2.csv" # Replace with the actual path to your training dataset CSV file

# Read the CSV file
train_df = pd.read_csv(train_csv_file, encoding='ISO-8859-1')

# Extract class names and convert them to indices
class_names = train_df['Species'].unique()
class_indices = {class_name: i for i, class_name in enumerate(class_names)}

# Convert class names to indices in the DataFrame
train_df['Class Index'] = train_df['Species'].map(class_indices)

# Calculate class weights based on class frequencies
class_weights = compute_class_weight("balanced", classes=np.unique(train_df['Class Index']), y=train_df['Class Index'])

# Normalize class weights to sum to 1
class_weights /= np.sum(class_weights)

# Create the alpha tensor
alpha_tensor = tf.constant(class_weights, dtype=tf.float32)

# Convert alpha tensor to a NumPy array
alpha_array = alpha_tensor.numpy()

# Print alpha values
print("Alpha values:")
for class_idx, alpha in enumerate(alpha_array):
    print(f"Class {class_idx}: {alpha}")

# Use alpha_array with CategoricalFocalCrossentropy
categorical_focal_loss = tf.keras.losses.CategoricalFocalCrossentropy(alpha=alpha_array, gamma=3)

Alpha values:
Class 0: 0.00036313605960458517
Class 1: 0.0006911298842169344
Class 2: 0.00013064040103927255
Class 3: 0.0001575369678903371
Class 4: 0.00037587765837088227
Class 5: 8.570010686526075e-05
Class 6: 0.0002856670180335641
Class 7: 0.0001575369678903371
Class 8: 0.0014283350901678205
Class 9: 0.00036939702113159
Class 10: 8.927094313548878e-05
Class 11: 0.0003512299444992095
Class 12: 0.0005493596545420587
Class 13: 8.570010686526075e-05
Class 14: 0.00030176094151102006
Class 15: 0.00014878490765113384
Class 16: 8.570010686526075e-05
Class 17: 0.0006695321062579751
Class 18: 0.0013390642125159502
Class 19: 8.570010686526075e-05
Class 20: 0.00014878490765113384
Class 21: 0.00015638706099707633
Class 22: 0.00014188759087119251
Class 23: 0.00010350254160584882
Class 24: 0.0006695321062579751
Class 25: 0.0006121436017565429
Class 26: 0.0021425026934593916
Class 27: 0.00015088047075551003
Class 28: 0.00045585163752548397
Class 29: 0.00011217291466891766
Class 30: 6.53202005196362

2024-03-23 18:54:54.791302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 18:54:54.825502: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 18:54:54.825568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 18:54:54.827232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 18:54:54.827301: I external/local_xla/xla/stream_executor

In [12]:
#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)

#clr
lr_schedule = tfa.optimizers.TriangularCyclicalLearningRate(
    initial_learning_rate=1e-3,
    maximal_learning_rate=1e-2,
    step_size=1504,
    scale_mode="cycle",
    name="CyclicScheduler")

In [13]:
model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(240,240,3), pooling=None, include_preprocessing=False)
model.trainable=False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(588, activation="softmax", name="pred")(x)
model = keras.Model(model.inputs, outputs, name="EfficientNet")
for layer in model.layers[:-1]: #freezing our low level layer weights
  layer.trainable = False
max_lr = 1e-2 # Adjust as needed
optimizer = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=max_lr, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss=categorical_focal_loss, metrics=['accuracy'])

In [14]:
for i, layer in enumerate(model.layers):
  print(f"{i}, {layer.name} : {layer.trainable}")

0, input_2 : False
1, stem_conv : False
2, stem_bn : False
3, stem_activation : False
4, block1a_project_conv : False
5, block1a_project_bn : False
6, block1a_project_activation : False
7, block1a_add : False
8, block1b_project_conv : False
9, block1b_project_bn : False
10, block1b_project_activation : False
11, block1b_drop : False
12, block1b_add : False
13, block2a_expand_conv : False
14, block2a_expand_bn : False
15, block2a_expand_activation : False
16, block2a_project_conv : False
17, block2a_project_bn : False
18, block2b_expand_conv : False
19, block2b_expand_bn : False
20, block2b_expand_activation : False
21, block2b_project_conv : False
22, block2b_project_bn : False
23, block2b_drop : False
24, block2b_add : False
25, block2c_expand_conv : False
26, block2c_expand_bn : False
27, block2c_expand_activation : False
28, block2c_project_conv : False
29, block2c_project_bn : False
30, block2c_drop : False
31, block2c_add : False
32, block2d_expand_conv : False
33, block2d_expand_

In [5]:
# Specify the path for saving model checkpoints
checkpoint_filepath = "/home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)" #edit this path
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, save_best_only=True, verbose=1, monitor='val_accuracy', mode='max')

In [18]:
# Train the model using the image generator for training and validation
history = model.fit(
    trainGen,
    epochs=50,
    steps_per_epoch=len(trainGen),
    validation_data=valGen,
    validation_steps=len(valGen),
    callbacks=[model_checkpoint]  
)

Epoch 1/50
752/752 [==============================] - ETA: 0s - loss: 0.0038 - accuracy: 0.4199
Epoch 1: val_accuracy improved from -inf to 0.43873, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 536s 712ms/step - loss: 0.0038 - accuracy: 0.4199 - val_loss: 0.0046 - val_accuracy: 0.4387
Epoch 2/50
752/752 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.4648
Epoch 2: val_accuracy improved from 0.43873 to 0.55151, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 544s 723ms/step - loss: 0.0040 - accuracy: 0.4648 - val_loss: 0.0037 - val_accuracy: 0.5515
Epoch 3/50
752/752 [==============================] - ETA: 0s - loss: 0.0023 - accuracy: 0.5833
Epoch 3: val_accuracy improved from 0.55151 to 0.62783, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


...

752/752 [==============================] - 539s 717ms/step - loss: 0.0023 - accuracy: 0.7077 - val_loss: 0.0038 - val_accuracy: 0.7043
Epoch 36/50
752/752 [==============================] - ETA: 0s - loss: 0.0020 - accuracy: 0.7288
Epoch 36: val_accuracy did not improve from 0.70431
752/752 [==============================] - 508s 676ms/step - loss: 0.0020 - accuracy: 0.7288 - val_loss: 0.0040 - val_accuracy: 0.6771
Epoch 37/50
752/752 [==============================] - ETA: 0s - loss: 0.0027 - accuracy: 0.6742
Epoch 37: val_accuracy did not improve from 0.70431
752/752 [==============================] - 487s 648ms/step - loss: 0.0027 - accuracy: 0.6742 - val_loss: 0.0046 - val_accuracy: 0.6410
Epoch 38/50
752/752 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 0.6651
Epoch 38: val_accuracy did not improve from 0.70431
752/752 [==============================] - 491s 652ms/step - loss: 0.0031 - accuracy: 0.6651 - val_loss: 0.0042 - val_accuracy: 0.6649
Epoch 39/50
7

INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 519s 690ms/step - loss: 0.0022 - accuracy: 0.7192 - val_loss: 0.0039 - val_accuracy: 0.7056
Epoch 44/50
752/752 [==============================] - ETA: 0s - loss: 0.0020 - accuracy: 0.7363
Epoch 44: val_accuracy did not improve from 0.70561
752/752 [==============================] - 464s 617ms/step - loss: 0.0020 - accuracy: 0.7363 - val_loss: 0.0041 - val_accuracy: 0.6866
Epoch 45/50
752/752 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 0.6990
Epoch 45: val_accuracy did not improve from 0.70561
752/752 [==============================] - 490s 651ms/step - loss: 0.0026 - accuracy: 0.6990 - val_loss: 0.0046 - val_accuracy: 0.6477
Epoch 46/50
752/752 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 0.6820
Epoch 46: val_accuracy did not improve from 0.70561
752/752 [==============================] - 486s 646ms/step - loss: 0.0029 - accuracy: 0.6820 - val_loss: 0.0045 - val_accuracy: 0.6732
Epoch 47/50
7

INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 530s 705ms/step - loss: 0.0022 - accuracy: 0.7297 - val_loss: 0.0040 - val_accuracy: 0.7071
Epoch 48/50
752/752 [==============================] - ETA: 0s - loss: 0.0020 - accuracy: 0.7446
Epoch 48: val_accuracy did not improve from 0.70708
752/752 [==============================] - 468s 622ms/step - loss: 0.0020 - accuracy: 0.7446 - val_loss: 0.0043 - val_accuracy: 0.6972
Epoch 49/50
752/752 [==============================] - ETA: 0s - loss: 0.0027 - accuracy: 0.7020
Epoch 49: val_accuracy did not improve from 0.70708
752/752 [==============================] - 475s 631ms/step - loss: 0.0027 - accuracy: 0.7020 - val_loss: 0.0048 - val_accuracy: 0.6636
Epoch 50/50
752/752 [==============================] - ETA: 0s - loss: 0.0030 - accuracy: 0.6840
Epoch 50: val_accuracy did not improve from 0.70708
752/752 [==============================] - 470s 625ms/step - loss: 0.0030 - accuracy: 0.6840 - val_loss: 0.0043 - val_accuracy: 0.6817


In [19]:
#clr
lr_schedule = tfa.optimizers.TriangularCyclicalLearningRate(
    initial_learning_rate=1e-5,
    maximal_learning_rate=1e-4,
    step_size=6016,
    scale_mode="cycle",
    name="CyclicScheduler")

In [20]:
for layer in model.layers:
  layer.trainable = False
for layer in model.layers[-156:]:
      if not isinstance(layer, layers.BatchNormalization):
          layer.trainable = True

for i, layer in enumerate(model.layers):
  print(f"{i}, {layer.name} : {layer.trainable}")

optimizer = Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss=categorical_focal_loss, metrics=['accuracy'])

0, input_2 : False
1, stem_conv : False
2, stem_bn : False
3, stem_activation : False
4, block1a_project_conv : False
5, block1a_project_bn : False
6, block1a_project_activation : False
7, block1a_add : False
8, block1b_project_conv : False
9, block1b_project_bn : False
10, block1b_project_activation : False
11, block1b_drop : False
12, block1b_add : False
13, block2a_expand_conv : False
14, block2a_expand_bn : False
15, block2a_expand_activation : False
16, block2a_project_conv : False
17, block2a_project_bn : False
18, block2b_expand_conv : False
19, block2b_expand_bn : False
20, block2b_expand_activation : False
21, block2b_project_conv : False
22, block2b_project_bn : False
23, block2b_drop : False
24, block2b_add : False
25, block2c_expand_conv : False
26, block2c_expand_bn : False
27, block2c_expand_activation : False
28, block2c_project_conv : False
29, block2c_project_bn : False
30, block2c_drop : False
31, block2c_add : False
32, block2d_expand_conv : False
33, block2d_expand_

In [21]:
# Train the model using the image generator for training and validation
history = model.fit(
    trainGen,
    epochs=25,
    steps_per_epoch=len(trainGen),
    validation_data=valGen,
    validation_steps=len(valGen),
    callbacks=[model_checkpoint]  
)

Epoch 1/25


2024-03-23 06:46:56.389062: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNet/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


752/752 [==============================] - ETA: 0s - loss: 0.0016 - accuracy: 0.7487
Epoch 1: val_accuracy improved from 0.70708 to 0.71880, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 584s 744ms/step - loss: 0.0016 - accuracy: 0.7487 - val_loss: 0.0031 - val_accuracy: 0.7188
Epoch 2/25
752/752 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 0.7711
Epoch 2: val_accuracy improved from 0.71880 to 0.72889, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


...

752/752 [==============================] - 559s 743ms/step - loss: 2.1150e-04 - accuracy: 0.8812 - val_loss: 0.0014 - val_accuracy: 0.7977
Epoch 15/25
752/752 [==============================] - ETA: 0s - loss: 1.8106e-04 - accuracy: 0.8939
Epoch 15: val_accuracy improved from 0.79772 to 0.80000, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 568s 756ms/step - loss: 1.8106e-04 - accuracy: 0.8939 - val_loss: 0.0014 - val_accuracy: 0.8000
Epoch 16/25
752/752 [==============================] - ETA: 0s - loss: 1.6748e-04 - accuracy: 0.9009
Epoch 16: val_accuracy improved from 0.80000 to 0.80439, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 570s 758ms/step - loss: 1.6748e-04 - accuracy: 0.9009 - val_loss: 0.0014 - val_accuracy: 0.8044
Epoch 17/25
752/752 [==============================] - ETA: 0s - loss: 1.5361e-04 - accuracy: 0.9019
Epoch 17: val_accuracy did not improve from 0.80439
752/752 [==============================] - 505s 671ms/step - loss: 1.5361e-04 - accuracy: 0.9019 - val_loss: 0.0014 - val_accuracy: 0.8003
Epoch 18/25
752/752 [==============================] - ETA: 0s - loss: 1.6555e-04 - accuracy: 0.8978
Epoch 18: val_accuracy did not improve from 0.80439
752/752 [==============================] - 505s 672ms/step - loss: 1.6555e-04 - accuracy: 0.8978 - val_loss: 0.0013 - val_accuracy: 0.8011
Epoch 19/25
752/752 [==============================] - ETA: 0s - loss: 1.6335e-04 - accuracy: 0.8969
Epoch 19: val_accuracy did not improve from 0.80439
752/752 [==============================] - 508s 675ms/step - loss: 1.6335e-04 - accuracy: 0.8969 - val_loss: 0.0014 - val_ac

In [4]:
model = tf.keras.models.load_model('/home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)')

In [9]:
#clr
lr_schedule = tfa.optimizers.TriangularCyclicalLearningRate(
    initial_learning_rate=1e-5,
    maximal_learning_rate=1e-4,
    step_size=1504,
    scale_mode="cycle",
    name="CyclicScheduler")

In [10]:
# Train the model using the image generator for training and validation
history = model.fit(
    trainGen,
    epochs=10,
    steps_per_epoch=len(trainGen),
    validation_data=valGen,
    validation_steps=len(valGen),
    callbacks=[model_checkpoint]  
)

Epoch 1/10
752/752 [==============================] - ETA: 0s - loss: 1.7597e-04 - accuracy: 0.8973
Epoch 1: val_accuracy did not improve from 0.80163
752/752 [==============================] - 660s 877ms/step - loss: 1.7597e-04 - accuracy: 0.8973 - val_loss: 0.0014 - val_accuracy: 0.8011
Epoch 2/10
752/752 [==============================] - ETA: 0s - loss: 1.8539e-04 - accuracy: 0.8912
Epoch 2: val_accuracy did not improve from 0.80163
752/752 [==============================] - 696s 925ms/step - loss: 1.8539e-04 - accuracy: 0.8912 - val_loss: 0.0015 - val_accuracy: 0.7992
Epoch 3/10
752/752 [==============================] - ETA: 0s - loss: 1.8903e-04 - accuracy: 0.8896
Epoch 3: val_accuracy did not improve from 0.80163
752/752 [==============================] - 676s 899ms/step - loss: 1.8903e-04 - accuracy: 0.8896 - val_loss: 0.0013 - val_accuracy: 0.7943
Epoch 4/10
752/752 [==============================] - ETA: 0s - loss: 2.0195e-04 - accuracy: 0.8862
Epoch 4: val_accuracy did not 

INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 907s 1s/step - loss: 1.6367e-04 - accuracy: 0.8984 - val_loss: 0.0014 - val_accuracy: 0.8026
Epoch 10/10
752/752 [==============================] - ETA: 0s - loss: 1.3680e-04 - accuracy: 0.9071
Epoch 10: val_accuracy improved from 0.80260 to 0.80797, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 742s 987ms/step - loss: 1.3680e-04 - accuracy: 0.9071 - val_loss: 0.0013 - val_accuracy: 0.8080


In [11]:
# Train the model using the image generator for training and validation
history = model.fit(
    trainGen,
    epochs=15,
    steps_per_epoch=len(trainGen),
    validation_data=valGen,
    validation_steps=len(valGen),
    callbacks=[model_checkpoint]  
)

Epoch 1/15
752/752 [==============================] - ETA: 0s - loss: 1.1474e-04 - accuracy: 0.9164
Epoch 1: val_accuracy improved from 0.80797 to 0.81058, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 802s 1s/step - loss: 1.1474e-04 - accuracy: 0.9164 - val_loss: 0.0013 - val_accuracy: 0.8106
Epoch 2/15
752/752 [==============================] - ETA: 0s - loss: 1.0917e-04 - accuracy: 0.9206
Epoch 2: val_accuracy improved from 0.81058 to 0.81318, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 892s 1s/step - loss: 1.0917e-04 - accuracy: 0.9206 - val_loss: 0.0013 - val_accuracy: 0.8132
Epoch 3/15
752/752 [==============================] - ETA: 0s - loss: 9.6632e-05 - accuracy: 0.9272
Epoch 3: val_accuracy improved from 0.81318 to 0.81806, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 746s 991ms/step - loss: 9.6632e-05 - accuracy: 0.9272 - val_loss: 0.0013 - val_accuracy: 0.8181
Epoch 4/15
752/752 [==============================] - ETA: 0s - loss: 8.4546e-05 - accuracy: 0.9322
Epoch 4: val_accuracy improved from 0.81806 to 0.82197, saving model to /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)
INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


INFO:tensorflow:Assets written to: /home/halwa/fyp/Training/Checkpoints/ScratchEffNetV2S(588Species)/assets


752/752 [==============================] - 754s 1s/step - loss: 8.4546e-05 - accuracy: 0.9322 - val_loss: 0.0012 - val_accuracy: 0.8220
Epoch 5/15
752/752 [==============================] - ETA: 0s - loss: 8.0396e-05 - accuracy: 0.9349
Epoch 5: val_accuracy did not improve from 0.82197
752/752 [==============================] - 751s 999ms/step - loss: 8.0396e-05 - accuracy: 0.9349 - val_loss: 0.0012 - val_accuracy: 0.8186
Epoch 6/15
752/752 [==============================] - ETA: 0s - loss: 7.9622e-05 - accuracy: 0.9337
Epoch 6: val_accuracy did not improve from 0.82197
752/752 [==============================] - 735s 977ms/step - loss: 7.9622e-05 - accuracy: 0.9337 - val_loss: 0.0013 - val_accuracy: 0.8210
Epoch 7/15
752/752 [==============================] - ETA: 0s - loss: 9.2216e-05 - accuracy: 0.9298
Epoch 7: val_accuracy did not improve from 0.82197
752/752 [==============================] - 735s 977ms/step - loss: 9.2216e-05 - accuracy: 0.9298 - val_loss: 0.0013 - val_accuracy: 0